In [9]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
from IPython.core.display import HTML

url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
wiki_page = requests.get(url)
soup = BeautifulSoup(wiki_page.content,'lxml')
table = soup.find_all('table')[0]
df = pd.read_html(str(table))[0]
df

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [10]:
df_drop = df[df.Borough != 'Not assigned']
df_drop

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [14]:
df_drop.shape

(103, 3)

# Answer to Dataframe question 

In [42]:
df_drop.head(8)

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills


In [15]:
ll = pd.read_csv('Geospatial_Coordinates.csv')
ll.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


# Answer to 'Geocoder' question

In [45]:
df_combine = df_drop.merge(ll,on='Postal Code')
df_combine.head(12)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [19]:
import folium
latitude = 43.651070
longitude = -79.347015
toronto_map = folium.Map(location=[latitude,longitude],zoom_start=11)

In [20]:
for lat, lng, borough, neighborhood in zip(df_combine['Latitude'], df_combine['Longitude'], df_combine['Borough'], df_combine['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)
    
HTML(toronto_map._repr_html_())

In [21]:
client_id = "AIQD5MZXYNYNSTKVKBQRIKKXENLZL05W2YIW2VB2OWORFO5G"
client_secret = "CXW1UPP3QNP130TH444GV2QJQAXZBC0GP5NEFWJODMALEGMB"
version = "20200605"

In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):           
     
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            client_id, 
            client_secret, 
            version, 
            lat, 
            lng, 
            radius, 
            10)
            
        
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
     
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [23]:
venues = getNearbyVenues(names=df_combine['Neighborhood'],latitudes=df_combine.Latitude,longitudes=df_combine.Longitude)

In [24]:
venues.shape

(686, 7)

In [44]:
venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


In [26]:
venues.groupby('Neighborhood').Venue.count().sort_values(ascending=False)

Neighborhood
Downsview                                                                                                                                 16
Don Mills                                                                                                                                 14
Parkview Hill, Woodbine Gardens                                                                                                           10
Central Bay Street                                                                                                                        10
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport                                10
                                                                                                                                          ..
Rouge Hill, Port Union, Highland Creek                                                                                                     1


In [27]:
venues['Venue Category'].nunique()

176

In [28]:
venues['Venue Category'].value_counts()

Coffee Shop        49
Café               32
Park               29
Restaurant         26
Pizza Place        21
                   ..
BBQ Joint           1
Medical Center      1
Shopping Plaza      1
Curling Ice         1
College Stadium     1
Name: Venue Category, Length: 176, dtype: int64

In [29]:
toronto_onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

toronto_onehot['Neighborhood'] = venues['Neighborhood']

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Terminal,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,...,Toy / Game Store,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,"Willowdale, Willowdale East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
90,"Willowdale, Willowdale West",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
91,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
92,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [31]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Latin American Restaurant,Skating Rink,Clothing Store,Breakfast Spot,Dog Run,Falafel Restaurant,Event Space,Electronics Store,Eastern European Restaurant
1,"Alderwood, Long Branch",Pizza Place,Pharmacy,Gym,Coffee Shop,Dance Studio,Athletics & Sports,Pub,Sandwich Place,Diner,Eastern European Restaurant
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Bridal Shop,Deli / Bodega,Diner,Ice Cream Shop,Middle Eastern Restaurant,Restaurant,Sushi Restaurant,Yoga Studio
3,Bayview Village,Café,Japanese Restaurant,Bank,Chinese Restaurant,Yoga Studio,Falafel Restaurant,Event Space,Electronics Store,Eastern European Restaurant,Drugstore
4,"Bedford Park, Lawrence Manor East",Comfort Food Restaurant,Pub,Juice Bar,Italian Restaurant,Sushi Restaurant,Indian Restaurant,Restaurant,Thai Restaurant,Café,Coffee Shop


In [32]:
from sklearn.cluster import KMeans
k = 5
toronto_cluster = toronto_grouped.drop('Neighborhood',axis=1)

model = KMeans(n_clusters=k,random_state=0)
model.fit(toronto_cluster)
model.labels_[0:10]

array([0, 2, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [33]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', model.labels_)

toronto_merged = df_combine
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'),on='Neighborhood')
toronto_merged

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,1.0,Fast Food Restaurant,Park,Food & Drink Shop,Discount Store,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
1,M4A,North York,Victoria Village,43.725882,-79.315572,2.0,Pizza Place,Coffee Shop,Portuguese Restaurant,Hockey Arena,Diner,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0.0,Breakfast Spot,Gym / Fitness Center,Historic Site,Coffee Shop,Restaurant,Distribution Center,Bakery,Spa,Park,Dessert Shop
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,0.0,Clothing Store,Accessories Store,Boutique,Furniture / Home Store,Event Space,Coffee Shop,Women's Store,Vietnamese Restaurant,Field,Farmers Market
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0.0,Yoga Studio,Sushi Restaurant,Creperie,Coffee Shop,Distribution Center,Mexican Restaurant,Beer Bar,Italian Restaurant,Park,Arts & Crafts Store
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944,1.0,River,Park,Pool,Dessert Shop,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,0.0,Park,Theme Restaurant,Bubble Tea Shop,Burger Joint,Beer Bar,Restaurant,Mexican Restaurant,Ramen Restaurant,Breakfast Spot,Dance Studio
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,0.0,Pizza Place,Auto Workshop,Comic Shop,Restaurant,Burrito Place,Skate Park,Brewery,Fast Food Restaurant,Farmers Market,Garden Center
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509,3.0,Baseball Field,Yoga Studio,Fast Food Restaurant,Falafel Restaurant,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run


In [34]:
toronto_merged.dropna(subset=['Cluster Labels'],inplace=True)

In [35]:

import matplotlib.cm as cm
import matplotlib.colors as colors

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)


x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]


markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
HTML(map_clusters._repr_html_())

In [36]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0]

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0.0,Breakfast Spot,Gym / Fitness Center,Historic Site,Coffee Shop,Restaurant,Distribution Center,Bakery,Spa,Park,Dessert Shop
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,0.0,Clothing Store,Accessories Store,Boutique,Furniture / Home Store,Event Space,Coffee Shop,Women's Store,Vietnamese Restaurant,Field,Farmers Market
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0.0,Yoga Studio,Sushi Restaurant,Creperie,Coffee Shop,Distribution Center,Mexican Restaurant,Beer Bar,Italian Restaurant,Park,Arts & Crafts Store
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,0.0,Fast Food Restaurant,Yoga Studio,Discount Store,Falafel Restaurant,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
7,M3B,North York,Don Mills,43.745906,-79.352188,0.0,Japanese Restaurant,Gym / Fitness Center,Italian Restaurant,Caribbean Restaurant,Café,Clothing Store,Discount Store,Coffee Shop,Sporting Goods Shop,Art Gallery
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675,0.0,Café,Indian Restaurant,General Entertainment,Jewelry Store,Diner,Japanese Restaurant,Italian Restaurant,Restaurant,Bakery,Dog Run
97,M5X,Downtown Toronto,"First Canadian Place, Underground city",43.648429,-79.382280,0.0,Coffee Shop,Restaurant,Café,Gym / Fitness Center,Bakery,Steakhouse,Gym,Art Gallery,Field,Farmers Market
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,0.0,Park,Theme Restaurant,Bubble Tea Shop,Burger Joint,Beer Bar,Restaurant,Mexican Restaurant,Ramen Restaurant,Breakfast Spot,Dance Studio
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,0.0,Pizza Place,Auto Workshop,Comic Shop,Restaurant,Burrito Place,Skate Park,Brewery,Fast Food Restaurant,Farmers Market,Garden Center


In [37]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1]

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,1.0,Fast Food Restaurant,Park,Food & Drink Shop,Discount Store,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
21,M6E,York,Caledonia-Fairbanks,43.689026,-79.453512,1.0,Park,Women's Store,Pool,Yoga Studio,Diner,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
35,M4J,East York,"East Toronto, Broadview North (Old East York)",43.685347,-79.338106,1.0,Convenience Store,Park,Discount Store,Falafel Restaurant,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
49,M6L,North York,"North Park, Maple Leaf Park, Upwood Park",43.713756,-79.490074,1.0,Construction & Landscaping,Park,Basketball Court,Bakery,Distribution Center,Falafel Restaurant,Event Space,Electronics Store,Eastern European Restaurant,Drugstore
61,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,1.0,Park,Swim School,Bus Line,Yoga Studio,Discount Store,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
64,M9N,York,Weston,43.706876,-79.518188,1.0,Convenience Store,Park,Discount Store,Falafel Restaurant,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
66,M2P,North York,York Mills West,43.752758,-79.400049,1.0,Park,Construction & Landscaping,Convenience Store,Golf Course,Falafel Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
68,M5P,Central Toronto,"Forest Hill North & West, Forest Hill Road Park",43.696948,-79.411307,1.0,Park,Jewelry Store,Trail,Sushi Restaurant,Yoga Studio,Discount Store,Event Space,Electronics Store,Eastern European Restaurant,Drugstore
83,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,1.0,Restaurant,Park,Trail,Diner,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
85,M1V,Scarborough,"Milliken, Agincourt North, Steeles East, L'Amo...",43.815252,-79.284577,1.0,Park,Playground,Coffee Shop,Bakery,Yoga Studio,Discount Store,Event Space,Electronics Store,Eastern European Restaurant,Drugstore


In [38]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2]

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,M4A,North York,Victoria Village,43.725882,-79.315572,2.0,Pizza Place,Coffee Shop,Portuguese Restaurant,Hockey Arena,Diner,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,2.0,Pizza Place,Athletics & Sports,Gastropub,Intersection,Fast Food Restaurant,Pet Store,Pharmacy,Bank,Gym / Fitness Center,Asian Restaurant
10,M6B,North York,Glencairn,43.709577,-79.445073,2.0,Pizza Place,Japanese Restaurant,Pub,Asian Restaurant,Discount Store,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
50,M9L,North York,Humber Summit,43.756303,-79.565963,2.0,Pizza Place,Furniture / Home Store,Convenience Store,Cosmetics Shop,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
67,M4P,Central Toronto,Davisville North,43.712751,-79.390197,2.0,Gym / Fitness Center,Breakfast Spot,Hotel,Food & Drink Shop,Department Store,Park,Sandwich Place,Pizza Place,Arts & Crafts Store,Cosmetics Shop
70,M9P,Etobicoke,Westmount,43.696319,-79.532242,2.0,Pizza Place,Discount Store,Coffee Shop,Middle Eastern Restaurant,Sandwich Place,Chinese Restaurant,Intersection,Eastern European Restaurant,Drugstore,Donut Shop
72,M2R,North York,"Willowdale, Willowdale West",43.782736,-79.442259,2.0,Pizza Place,Bank,Grocery Store,Coffee Shop,Pharmacy,Gay Bar,Electronics Store,Drugstore,Donut Shop,Dog Run
77,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724,2.0,Pizza Place,Sandwich Place,Bus Line,Mobile Phone Shop,Cuban Restaurant,Curling Ice,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
89,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437,2.0,Grocery Store,Pizza Place,Beer Store,Fried Chicken Joint,Fast Food Restaurant,Liquor Store,Sandwich Place,Pharmacy,Arts & Crafts Store,Airport Food Court
90,M1W,Scarborough,"Steeles West, L'Amoreaux West",43.799525,-79.318389,2.0,Chinese Restaurant,Pizza Place,Bank,Pharmacy,Grocery Store,Coffee Shop,Sandwich Place,Fast Food Restaurant,Breakfast Spot,Garden


In [39]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3]

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
57,M9M,North York,"Humberlea, Emery",43.724766,-79.532242,3.0,Baseball Field,Yoga Studio,Fast Food Restaurant,Falafel Restaurant,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509,3.0,Baseball Field,Yoga Studio,Fast Food Restaurant,Falafel Restaurant,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run


In [40]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4]

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,4.0,Bar,Yoga Studio,Fast Food Restaurant,Falafel Restaurant,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
